In [148]:
import os
from datetime import date
from datetime import datetime as dt
import shutil
import pandas as pd

## Scratchpad

In [230]:
today = str(date.today()).replace('-','_')
dropzone = '/Users/karim/Documents/GitHub/webscraping/dev/clickup/dropzone/'
archive = '/Users/karim/Documents/GitHub/webscraping/dev/clickup/archive/'


In [150]:
#os.getcwd()

In [231]:
# rename file
def process_dropzone(dropzone):
    filename = max([f for f in os.listdir(dropzone)], key=lambda xa :   os.path.getctime(os.path.join(dropzone,xa)))
    os.rename(os.path.join(dropzone,filename),os.path.join(dropzone, f'clickup_{today}.csv'))
    new_name = max([f for f in os.listdir(dropzone)], key=lambda xa :   os.path.getctime(os.path.join(dropzone,xa)))
    shutil.move(f"/Users/karim/Documents/GitHub/webscraping/dev/clickup/dropzone/{new_name}", f"/Users/karim/Documents/GitHub/webscraping/dev/clickup/archive/{new_name}")

In [233]:
#process two files
process_dropzone(dropzone)


In [234]:
#get latest two files
sorted_lsit = sorted([f for f in os.listdir(archive)], key=lambda xa :   os.path.getctime(os.path.join(archive,xa)))
print(sorted_lsit[:2])

['clickup_2022_01_15.csv', 'clickup_2022_01_15 copy.csv']


In [235]:
print(archive+sorted_lsit[0])


/Users/karim/Documents/GitHub/webscraping/dev/clickup/archive/clickup_2022_01_15.csv


In [281]:
#load files from archive
df1 = pd.read_csv(archive+sorted_lsit[0])
df2 = pd.read_csv(archive+sorted_lsit[1])


In [282]:
#filter down on account and opportunity list
df1 = df1[df1['List Name'].eq('Accounts & Opportunities')]
#smaller set of columns
df1 = df1[['Task ID', ' Task Name', 'Status', 'Date Created',
       'Date Created Text','Assignees', 'Tags']]
#convert date column to datatime format
df1['Current_Date'] = date.today()
df1['Current_Date'] = pd.to_datetime(df1['Current_Date'])

In [283]:
#filter down on account and opportunity list
df2 = df2[df2['List Name'].eq('Accounts & Opportunities')]
#smaller set of columns
df2 = df2[['Task ID', ' Task Name', 'Status', 'Date Created',
       'Date Created Text','Assignees', 'Tags']]
#convert date column to datatime format
df2['Date Created Text'] = df2['Date Created Text'].str[:10]
df2['Date Created Text'] = pd.to_datetime(df2['Date Created Text'])

In [284]:
df3 = df1.merge(df2,how='left',left_on='Task ID',right_on='Task ID',suffixes=['_current','_previous'])

In [285]:
df3.columns

Index(['Task ID', ' Task Name_current', 'Status_current',
       'Date Created_current', 'Date Created Text_current',
       'Assignees_current', 'Tags_current', 'Current_Date',
       ' Task Name_previous', 'Status_previous', 'Date Created_previous',
       'Date Created Text_previous', 'Assignees_previous', 'Tags_previous'],
      dtype='object')

## Commit

In [286]:
df3 = df3[['Task ID','Assignees_current',' Task Name_current','Status_current','Status_previous','Current_Date','Date Created Text_current',
       'Date Created Text_previous']]
#, 'Comments','Assigned Comments'

df3.head()

,Task ID,Assignees_current,Task Name_current,Status_current,Status_previous,Current_Date,Date Created Text_current,Date Created Text_previous
0,272at2e,[],VICTORIA MELINDA MILLS BROUILLET,intro call,intro call,2022-01-15,"12/29/2021, 1:15:53 PM GMT+4",2021-12-29
1,272at1w,[],NOUMAN SAKHAWAT SAKHAWAT KHAN,closed [lost],closed [lost],2022-01-15,"12/29/2021, 1:15:45 PM GMT+4",2021-12-29
2,272at34,[],SAFIYA SULAIMAN RASHID MATAR ALMATROOSHI,intro call,intro call,2022-01-15,"12/29/2021, 1:16:03 PM GMT+4",2021-12-29
3,272aryb,[],MUTABAR KHAIDARZODA,unqualified prospect,unqualified prospect,2022-01-15,"12/29/2021, 1:15:01 PM GMT+4",2021-12-29
4,272aryk,[],SANA SALAMAT SALAMAT JAN,unqualified prospect,unqualified prospect,2022-01-15,"12/29/2021, 1:15:06 PM GMT+4",2021-12-29


In [287]:
df3.dtypes

Task ID                               object
Assignees_current                     object
 Task Name_current                    object
Status_current                        object
Status_previous                       object
Current_Date                  datetime64[ns]
Date Created Text_current             object
Date Created Text_previous    datetime64[ns]
dtype: object

In [288]:
df3['DIFF']= df3['Current_Date']-df3['Date Created Text_previous'] 

In [289]:
df3

,Task ID,Assignees_current,Task Name_current,Status_current,Status_previous,Current_Date,Date Created Text_current,Date Created Text_previous,DIFF
0,272at2e,[],VICTORIA MELINDA MILLS BROUILLET,intro call,intro call,2022-01-15,"12/29/2021, 1:15:53 PM GMT+4",2021-12-29,17 days
1,272at1w,[],NOUMAN SAKHAWAT SAKHAWAT KHAN,closed [lost],closed [lost],2022-01-15,"12/29/2021, 1:15:45 PM GMT+4",2021-12-29,17 days
2,272at34,[],SAFIYA SULAIMAN RASHID MATAR ALMATROOSHI,intro call,intro call,2022-01-15,"12/29/2021, 1:16:03 PM GMT+4",2021-12-29,17 days
3,272aryb,[],MUTABAR KHAIDARZODA,unqualified prospect,unqualified prospect,2022-01-15,"12/29/2021, 1:15:01 PM GMT+4",2021-12-29,17 days
4,272aryk,[],SANA SALAMAT SALAMAT JAN,unqualified prospect,unqualified prospect,2022-01-15,"12/29/2021, 1:15:06 PM GMT+4",2021-12-29,17 days
...,...,...,...,...,...,...,...,...,...
143,20dc639,[],SHAIMA SAEED SAIF RASHID ALNAQBI,intro call,intro call,2022-01-15,"1/2/2022, 1:02:15 PM GMT+4",2022-01-02,13 days
144,272at1c,[],HOUSSAM MERHI,intro call,intro call,2022-01-15,"12/29/2021, 1:15:39 PM GMT+4",2021-12-29,17 days
145,272arze,[],YULIA NORMUKHAMEDOVA,unqualified prospect,unqualified prospect,2022-01-15,"12/29/2021, 1:15:18 PM GMT+4",2021-12-29,17 days
146,272arz9,[],ABDUL ANAS ABDUL HAMEED SAIDUTTY KOTTAYIL ABDU...,intro call,intro call,2022-01-15,"12/29/2021, 1:15:16 PM GMT+4",2021-12-29,17 days


In [290]:
#Ticket havent moved
df4 = df3[df3['Status_current']==df3['Status_current']]
df4

,Task ID,Assignees_current,Task Name_current,Status_current,Status_previous,Current_Date,Date Created Text_current,Date Created Text_previous,DIFF
0,272at2e,[],VICTORIA MELINDA MILLS BROUILLET,intro call,intro call,2022-01-15,"12/29/2021, 1:15:53 PM GMT+4",2021-12-29,17 days
1,272at1w,[],NOUMAN SAKHAWAT SAKHAWAT KHAN,closed [lost],closed [lost],2022-01-15,"12/29/2021, 1:15:45 PM GMT+4",2021-12-29,17 days
2,272at34,[],SAFIYA SULAIMAN RASHID MATAR ALMATROOSHI,intro call,intro call,2022-01-15,"12/29/2021, 1:16:03 PM GMT+4",2021-12-29,17 days
3,272aryb,[],MUTABAR KHAIDARZODA,unqualified prospect,unqualified prospect,2022-01-15,"12/29/2021, 1:15:01 PM GMT+4",2021-12-29,17 days
4,272aryk,[],SANA SALAMAT SALAMAT JAN,unqualified prospect,unqualified prospect,2022-01-15,"12/29/2021, 1:15:06 PM GMT+4",2021-12-29,17 days
...,...,...,...,...,...,...,...,...,...
143,20dc639,[],SHAIMA SAEED SAIF RASHID ALNAQBI,intro call,intro call,2022-01-15,"1/2/2022, 1:02:15 PM GMT+4",2022-01-02,13 days
144,272at1c,[],HOUSSAM MERHI,intro call,intro call,2022-01-15,"12/29/2021, 1:15:39 PM GMT+4",2021-12-29,17 days
145,272arze,[],YULIA NORMUKHAMEDOVA,unqualified prospect,unqualified prospect,2022-01-15,"12/29/2021, 1:15:18 PM GMT+4",2021-12-29,17 days
146,272arz9,[],ABDUL ANAS ABDUL HAMEED SAIDUTTY KOTTAYIL ABDU...,intro call,intro call,2022-01-15,"12/29/2021, 1:15:16 PM GMT+4",2021-12-29,17 days


In [291]:
df4.sort_values(by='DIFF',ascending=False)

,Task ID,Assignees_current,Task Name_current,Status_current,Status_previous,Current_Date,Date Created Text_current,Date Created Text_previous,DIFF
29,1xhv104,[],Kivanc Kilicer - Business set-up inquiry,closed [lost],closed [lost],2022-01-15,"12/14/2021, 9:25:22 AM GMT+4",2021-12-14,32 days
33,1zwbvba,[],Mira Brandli - 1 Properties - Inquiry for Will...,meeting scheduled,meeting scheduled,2022-01-15,"12/27/2021, 1:34:28 PM GMT+4",2021-12-27,19 days
59,20204qq,[Ese],Azor Shams - Inquiry to make case against cons...,in-development,in-development,2022-01-15,"12/28/2021, 5:35:18 PM GMT+4",2021-12-28,18 days
0,272at2e,[],VICTORIA MELINDA MILLS BROUILLET,intro call,intro call,2022-01-15,"12/29/2021, 1:15:53 PM GMT+4",2021-12-29,17 days
99,272arz0,[],OFELIA MANI GADICHO,intro call,intro call,2022-01-15,"12/29/2021, 1:15:13 PM GMT+4",2021-12-29,17 days
...,...,...,...,...,...,...,...,...,...
110,20dcc2u,[],MOHAMED SALAH ABDELWAHAB CHAHEN,intro call,intro call,2022-01-15,"1/2/2022, 1:43:01 PM GMT+4",2022-01-02,13 days
140,20djbvn,[],MOHAMED ALMAS HARIB HARIB ALFALAHI,meeting scheduled,meeting scheduled,2022-01-15,"1/3/2022, 2:07:40 PM GMT+4",2022-01-03,12 days
68,20djaxw,[],MUDASIR MUSHTAQ MUSHTAQ AHMED WANI,closed [lost],closed [lost],2022-01-15,"1/3/2022, 2:04:26 PM GMT+4",2022-01-03,12 days
126,20djaf5,[],SALEEM RAFIQ RAFIQ HAJI AZIZ AHMED AHMED,intro call,intro call,2022-01-15,"1/3/2022, 2:00:57 PM GMT+4",2022-01-03,12 days


In [297]:
df4.columns

Index(['Task ID', 'Assignees_current', ' Task Name_current', 'Status_current',
       'Status_previous', 'Current_Date', 'Date Created Text_current',
       'Date Created Text_previous', 'DIFF'],
      dtype='object')

In [304]:
pd.pivot_table(df4,index='Status_current',columns='Status_previous',values='DIFF',aggfunc='count')

Status_previous,agreement sent,closed [lost],in-development,intro call,meeting scheduled,unqualified prospect
Status_current,,,,,,
agreement sent,4.0,NaN,NaN,NaN,NaN,NaN
closed [lost],NaN,38.0,NaN,NaN,NaN,NaN
in-development,NaN,NaN,6.0,NaN,NaN,NaN
intro call,NaN,NaN,NaN,53.0,NaN,NaN
meeting scheduled,NaN,NaN,NaN,NaN,2.0,NaN
unqualified prospect,NaN,NaN,NaN,NaN,NaN,45.0


In [305]:
pd.pivot_table(df4,index='Status_current',columns='Status_previous',values='DIFF',aggfunc='mean')

Status_previous,agreement sent,closed [lost],in-development,intro call,meeting scheduled,unqualified prospect
Status_current,,,,,,
agreement sent,16 days,NaT,NaT,NaT,NaT,NaT
closed [lost],NaT,17 days 05:41:03.157894736,NaT,NaT,NaT,NaT
in-development,NaT,NaT,17 days 04:00:00,NaT,NaT,NaT
intro call,NaT,NaT,NaT,16 days 14:29:26.037735849,NaT,NaT
meeting scheduled,NaT,NaT,NaT,NaT,15 days 12:00:00,NaT
unqualified prospect,NaT,NaT,NaT,NaT,NaT,17 days
